In [1]:
import geopandas as gp
import numpy as np
gdf=gp.read_file("/Users/thomasjhojlunddodd/documents/data_analysis_malcolm/SLE_KenemaRadius/SLE_KenemaRadius_OSMBuildings.geojson")
gdf["x"] = gdf.centroid.map(lambda p: p.x)
gdf["y"] = gdf.centroid.map(lambda p: p.y)
gdf1 = gdf[['x','y']]
points = gdf1.to_numpy()

from scipy.spatial import Delaunay
tri = Delaunay(points)

import matplotlib.pyplot as plt
plt.triplot(points[:,0], points[:,1], tri.simplices.copy())
plt.plot(points[:,0], points[:,1], 'o')

# Get Delaunay triagulation of coordinates
from scipy.spatial import Delaunay
tri = Delaunay(points)
indices = tri.vertex_neighbor_vertices[0]
indptr = tri.vertex_neighbor_vertices[1]

#Instantiate dictionary to hold neighbors of each point and data frame to hold distances between neighbours.
import pandas as pd
neighbors = {}
locations = {}
distances = pd.DataFrame(columns=["source", "dest", "distance"])

#Get dictionary of neighbours of all points and a dictionary of locations of all points
no_points = gdf1.shape[0]  # gives number of row count

for k in range (0, no_points):
    neighbors[k] = indptr[indices[k]:indices[k+1]]
    locations[k] = points[k][0], points[k][1]
    
#Get distances between delaunay neighbors
import math
for key, values in neighbors.items():
    for value in values:
        coord_1 = points[key]
        coord_2 = points[value]
        dist = math.sqrt(((coord_1[0]-coord_2[0])**2)+((coord_1[1]-coord_2[1])**2))
        #dist = haversine_dist_from_coords(coord_1[1], coord_1[0], coord_2[1], coord_2[0])
        distances = distances.append({"source":key, "dest":value, "distance":dist}, ignore_index=True)

import networkx as nx
G = nx.Graph()

for index, row in distances.iterrows():
    G.add_edge(row['source'], row['dest'], weight=row['distance'])
T = nx.minimum_spanning_tree(G)

nx.draw(T,pos=locations,node_size=15)

import shapely
list = []

for x in T.edges:
    list.append(shapely.geometry.LineString([locations[x[0]],locations[x[1]]]))

Tgdf= gp.GeoDataFrame(geometry = list)
Tgdf.to_file("/Users/thomasjhojlunddodd/documents/data_analysis_malcolm/useful.shp")

/Users/thomasjhojlunddodd/anaconda3/lib/python3.7/site-packages/networkx/drawing/nx_pylab.py:579: MatplotlibDeprecationWarning: 
The iterable function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use np.iterable instead.
  if not cb.iterable(width):
